In [61]:
from cps_data_agg import (process_cps_data)
file_path = r'C:\Users\sneha\Downloads\cps_00006.csv.gz'
data = process_cps_data(file_path)

data

,STATEFIP,YEAR,Residuals,STATEFIP_2,STATEFIP_4,STATEFIP_5,STATEFIP_6,STATEFIP_8,STATEFIP_9,STATEFIP_10,...,YEAR_1991,YEAR_1992,YEAR_1993,YEAR_1994,YEAR_1995,YEAR_1996,YEAR_1997,YEAR_1998,YEAR_1999,YEAR_2000
0,1,1980,-0.782133,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1981,-0.472124,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1982,-0.589845,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1983,-0.519442,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1984,-0.429119,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,56,1996,0.013756,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1046,56,1997,-0.012077,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1047,56,1998,0.066763,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1048,56,1999,0.122135,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
!pip install statsmodels.stats.diagnostic

ERROR: Could not find a version that satisfies the requirement statsmodels.stats.diagnostic (from versions: none)
ERROR: No matching distribution found for statsmodels.stats.diagnostic


In [68]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS


beta1_estimates = []
reject_count = 0  # Counter for the number of rejections
alpha = 0.05  # Significance level
num_simulations = 400
num_individuals = 20
bias_values = []
squared_error_values = []
standard_error_values = []  # List to store standard errors
squared_error_values = []
standard_error_values = []  # List to store standard errors

true_beta1_value = 0


for _ in range(400):
    
    states = data['STATEFIP'].unique()
    

    # Randomly select half of the states to be in the treatment group
    
    treatment_states = np.random.choice(states, size=len(states)//2, replace=False)

    # Assign treatment year to each treatment state, staggered between 1985 and 1995
    treatment_years = np.random.choice(range(1985, 1995), size=len(treatment_states), replace=True)
    state_to_treatment_year = dict(zip(treatment_states, treatment_years))

    # Add a treatment column to the DataFrame
    data['TREATMENT'] = data.apply(lambda x: 1 if x['STATEFIP'] in treatment_states and x['YEAR'] >= state_to_treatment_year[x['STATEFIP']] else 0, axis=1)

    X = data[[ 'TREATMENT', 'STATEFIP_2',
       'STATEFIP_4', 'STATEFIP_5', 'STATEFIP_6', 'STATEFIP_8', 'STATEFIP_9',
       'STATEFIP_10', 'STATEFIP_12', 'STATEFIP_13', 'STATEFIP_15',
       'STATEFIP_16', 'STATEFIP_17', 'STATEFIP_18', 'STATEFIP_19',
       'STATEFIP_20', 'STATEFIP_21', 'STATEFIP_22', 'STATEFIP_23',
       'STATEFIP_24', 'STATEFIP_25', 'STATEFIP_26', 'STATEFIP_27',
       'STATEFIP_28', 'STATEFIP_29', 'STATEFIP_30', 'STATEFIP_31',
       'STATEFIP_32', 'STATEFIP_33', 'STATEFIP_34', 'STATEFIP_35',
       'STATEFIP_36', 'STATEFIP_37', 'STATEFIP_38', 'STATEFIP_39',
       'STATEFIP_40', 'STATEFIP_41', 'STATEFIP_42', 'STATEFIP_44',
       'STATEFIP_45', 'STATEFIP_46', 'STATEFIP_47', 'STATEFIP_48',
       'STATEFIP_49', 'STATEFIP_50', 'STATEFIP_51', 'STATEFIP_53',
       'STATEFIP_54', 'STATEFIP_55', 'STATEFIP_56',  'YEAR_1981', 'YEAR_1982',
       'YEAR_1983', 'YEAR_1984', 'YEAR_1985', 'YEAR_1986', 'YEAR_1987',
       'YEAR_1988', 'YEAR_1989', 'YEAR_1990', 'YEAR_1991', 'YEAR_1992',
       'YEAR_1993', 'YEAR_1994', 'YEAR_1995', 'YEAR_1996', 'YEAR_1997',
       'YEAR_1998', 'YEAR_1999', 'YEAR_2000']] # plus any other control variables
    
    Y = data['Residuals'] # Replace 'outcome' with your dependent variable
    # Randomly select half of the states to be in the treatment group
    
   
    
    initial_model = sm.GLS(Y, X)
    initial_results = initial_model.fit()
    ols_resid = initial_results.resid

    resid_fit = sm.OLS(np.asarray(ols_resid)[1:], sm.add_constant(np.asarray(ols_resid)[:-1])
     ).fit()

    rho = resid_fit.params[1]
    from scipy.linalg import toeplitz
    order = toeplitz(range(len(ols_resid)))

    sigma = rho ** order
    fgls_model = sm.GLS(Y, sm.add_constant(X), sigma= sigma)
    model = fgls_model.fit()
    

# Calculate the autocorrelation function of the residuals
    
    standard_error = model.bse['TREATMENT']
    standard_error_values.append(standard_error)

    bias = model.params['TREATMENT'] - true_beta1_value
    bias_values.append(bias)

    squared_error = (model.params['TREATMENT'] - true_beta1_value) ** 2
    squared_error_values.append(squared_error)

    # Check if null hypothesis for beta1 is rejected
    if model.pvalues['TREATMENT'] < alpha:
        reject_count += 1

    # Store the beta estimate
    beta1_estimates.append(model.params['TREATMENT'])


type1_error = reject_count / num_simulations
average_bias = np.mean(bias_values)
average_mse = np.mean(squared_error_values)
average_standard_error = np.mean(standard_error_values)

# Calculate the standard error of the distribution of beta
std_error_beta_distribution = np.std(beta1_estimates)
print(f"Number of times null hypothesis is rejected: {reject_count} out of {num_simulations} simulations")
print(f"Type 1 Error: {type1_error}")
print(f"Bias for Coefficient of Treatment (True Value = {true_beta1_value}): {average_bias}")
print(f"Average MSE for Coefficient of Treatment (True Value = {true_beta1_value}): {average_mse}")
print(f"Average Standard Error for Coefficient of Treatment: {average_standard_error}")
print(f"Standard Error of the Distribution of Beta: {std_error_beta_distribution}")


Number of times null hypothesis is rejected: 46 out of 400 simulations
Type 1 Error: 0.115
Bias for Coefficient of Treatment (True Value = 0): -0.0005211763005372429
Average MSE for Coefficient of Treatment (True Value = 0): 0.00024551876192743636
Average Standard Error for Coefficient of Treatment: 0.013145635288720084
Standard Error of the Distribution of Beta: 0.01566036836064831


In [52]:
13/200

0.065

In [66]:
treatment_states = np.random.choice(states, size=len(states)//2, replace=False)

    # Assign treatment year to each treatment state, staggered between 1985 and 1995
treatment_years = np.random.choice(range(1985, 1995), size=len(treatment_states), replace=True)
state_to_treatment_year = dict(zip(treatment_states, treatment_years))

    # Add a treatment column to the DataFrame
data['TREATMENT'] = data.apply(lambda x: 1 if x['STATEFIP'] in treatment_states and x['YEAR'] >= state_to_treatment_year[x['STATEFIP']] else 0, axis=1)

X = data[[ 'TREATMENT', 'STATEFIP_2',
       'STATEFIP_4', 'STATEFIP_5', 'STATEFIP_6', 'STATEFIP_8', 'STATEFIP_9',
       'STATEFIP_10', 'STATEFIP_12', 'STATEFIP_13', 'STATEFIP_15',
       'STATEFIP_16', 'STATEFIP_17', 'STATEFIP_18', 'STATEFIP_19',
       'STATEFIP_20', 'STATEFIP_21', 'STATEFIP_22', 'STATEFIP_23',
       'STATEFIP_24', 'STATEFIP_25', 'STATEFIP_26', 'STATEFIP_27',
       'STATEFIP_28', 'STATEFIP_29', 'STATEFIP_30', 'STATEFIP_31',
       'STATEFIP_32', 'STATEFIP_33', 'STATEFIP_34', 'STATEFIP_35',
       'STATEFIP_36', 'STATEFIP_37', 'STATEFIP_38', 'STATEFIP_39',
       'STATEFIP_40', 'STATEFIP_41', 'STATEFIP_42', 'STATEFIP_44',
       'STATEFIP_45', 'STATEFIP_46', 'STATEFIP_47', 'STATEFIP_48',
       'STATEFIP_49', 'STATEFIP_50', 'STATEFIP_51', 'STATEFIP_53',
       'STATEFIP_54', 'STATEFIP_55', 'STATEFIP_56',  'YEAR_1981', 'YEAR_1982',
       'YEAR_1983', 'YEAR_1984', 'YEAR_1985', 'YEAR_1986', 'YEAR_1987',
       'YEAR_1988', 'YEAR_1989', 'YEAR_1990', 'YEAR_1991', 'YEAR_1992',
       'YEAR_1993', 'YEAR_1994', 'YEAR_1995', 'YEAR_1996', 'YEAR_1997',
       'YEAR_1998', 'YEAR_1999', 'YEAR_2000']] # plus any other control variables
    
Y = data['Residuals'] # Replace 'outcome' with your dependent variable
    # Randomly select half of the states to be in the treatment group
    
initial_model = sm.GLS(Y, X)
initial_results = initial_model.fit()
ols_resid = initial_results.resid

resid_fit = sm.OLS(
    np.asarray(ols_resid)[1:], sm.add_constant(np.asarray(ols_resid)[:-1])
).fit()

rho = resid_fit.params[1]
from scipy.linalg import toeplitz
order = toeplitz(range(len(ols_resid)))

sigma = rho ** order
fgls_model = sm.GLS(Y, sm.add_constant(X), sigma= sigma)
model = fgls_model.fit()






In [43]:
print(resid_fit.tvalues[1])
print(resid_fit.pvalues[1])

21.200166889273525
2.8155295834738853e-83


In [44]:
resid_fit.params

array([-0.00418637,  0.53660831])

In [45]:
ols_resid

0      -0.782133
1      -0.381260
2      -0.580103
3      -0.582782
4      -0.540437
          ...   
1045    0.016976
1046   -0.044259
1047   -0.016196
1048   -0.040237
1049    0.008785
Length: 1050, dtype: float64

In [46]:
resid_fit = sm.OLS(
    np.asarray(ols_resid)[1:], sm.add_constant(np.asarray(ols_resid)[:-1])
).fit()
print(resid_fit.tvalues[1])
print(resid_fit.pvalues[1])


21.200166889273525
2.8155295834738853e-83


In [47]:

rho = resid_fit.params[1]


In [48]:
from scipy.linalg import toeplitz
order = toeplitz(range(len(ols_resid)))

sigma = rho ** order
fgls_model = sm.GLS(Y, sm.add_constant(X), sigma= sigma)
model = fgls_model.fit()
    


In [50]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:              Residuals   R-squared:                       0.927
Model:                            GLS   Adj. R-squared:                  0.922
Method:                 Least Squares   F-statistic:                     178.5
Date:                Tue, 26 Dec 2023   Prob (F-statistic):               0.00
Time:                        17:52:07   Log-Likelihood:                 1276.0
No. Observations:                1050   AIC:                            -2410.
Df Residuals:                     979   BIC:                            -2058.
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.7600      0.035    -21.936      0.000      -0.828      -0.692
TREATMENT       0.0135      0.013      1.025      0.306      -0.012       0.039
STATEFIP_2      0.2286      0.043      5.270      0.000       0.143       0.314
STATEFIP_4      0.1008      0.045      2.231      0.026       0.012       0.190
STATEFIP_5     -0.0857      0.045     -1.887      0.059      -0.175       0.003
STATEFIP_6      0.2410      0.046      5.218      0.000       0.150       0.332
STATEFIP_8      0.1023      0.046      2.241      0.025       0.013       0.192
STATEFIP_9      0.2838      0.045      6.249      0.000       0.195       0.373
STATEFIP_10     0.1588      0.046      3.451      0.001       0.069       0.249
STATEFIP_12     0.1142      0.045      2.514      0.012       0.025       0.203
STATEFIP_13     0.1564      0.045      3.442      0.001       0.067       0.245
STATEFIP_15     0.2136      0.046      4.649      0.000       0.123       0.304
STATEFIP_16    -0.2414      0.046     -5.278      0.000      -0.331      -0.152
STATEFIP_17     0.1838      0.046      4.010      0.000       0.094       0.274
STATEFIP_18     0.0289      0.046      0.630      0.529      -0.061       0.119
STATEFIP_19    -0.1023      0.046     -2.243      0.025      -0.192      -0.013
STATEFIP_20    -0.0125      0.045     -0.275      0.783      -0.102       0.077
STATEFIP_21    -0.0521      0.046     -1.124      0.261      -0.143       0.039
STATEFIP_22    -0.0192      0.046     -0.417      0.677      -0.110       0.071
STATEFIP_23    -0.0675      0.046     -1.472      0.141      -0.157       0.022
STATEFIP_24     0.3647      0.045      8.029      0.000       0.276       0.454
STATEFIP_25     0.2343      0.045      5.159      0.000       0.145       0.323
STATEFIP_26     0.0795      0.045      1.749      0.081      -0.010       0.169
STATEFIP_27     0.0327      0.046      0.706      0.480      -0.058       0.124
STATEFIP_28    -0.0670      0.045     -1.475      0.141      -0.156       0.022
STATEFIP_29     0.0683      0.045      1.504      0.133      -0.021       0.157
STATEFIP_30    -0.2441      0.045     -5.373      0.000      -0.333      -0.155
STATEFIP_31    -0.1541      0.045     -3.393      0.001      -0.243      -0.065
STATEFIP_32     0.2109      0.045      4.643      0.000       0.122       0.300
STATEFIP_33     0.1307      0.046      2.810      0.005       0.039       0.222
STATEFIP_34     0.2701      0.046      5.809      0.000       0.179       0.361
STATEFIP_35    -0.1025      0.046     -2.213      0.027      -0.193      -0.012
STATEFIP_36     0.2438      0.045      5.367      0.000       0.155       0.333
STATEFIP_37     0.1329      0.046      2.907      0.004       0.043       0.223
STATEFIP_38    -0.1864      0.045     -4.103      0.000      -0.276      -0.097
STATEFIP_39     0.0572      0.045      1.25

In [53]:
data

,STATEFIP,YEAR,Residuals,STATEFIP_2,STATEFIP_4,STATEFIP_5,STATEFIP_6,STATEFIP_8,STATEFIP_9,STATEFIP_10,...,YEAR_1992,YEAR_1993,YEAR_1994,YEAR_1995,YEAR_1996,YEAR_1997,YEAR_1998,YEAR_1999,YEAR_2000,TREATMENT
0,1,1980,-0.782133,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1981,-0.472124,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1982,-0.589845,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1983,-0.519442,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1984,-0.429119,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,56,1996,0.013756,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1046,56,1997,-0.012077,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1047,56,1998,0.066763,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1048,56,1999,0.122135,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [60]:
data_state = data.groupby('STATEFIP')

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

In [59]:
data.head()

,STATEFIP,YEAR,Residuals,STATEFIP_2,STATEFIP_4,STATEFIP_5,STATEFIP_6,STATEFIP_8,STATEFIP_9,STATEFIP_10,...,YEAR_1992,YEAR_1993,YEAR_1994,YEAR_1995,YEAR_1996,YEAR_1997,YEAR_1998,YEAR_1999,YEAR_2000,TREATMENT
0,1,1980,-0.782133,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1981,-0.472124,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1982,-0.589845,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1983,-0.519442,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1984,-0.429119,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,56,1980,-0.836900,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1030,56,1981,-0.590284,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1031,56,1982,-0.740115,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1032,56,1983,-0.676749,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm



treatment_states = np.random.choice(states, size=len(states)//2, replace=False)

    # Assign treatment year to each treatment state, staggered between 1985 and 1995
treatment_years = np.random.choice(range(1985, 1995), size=len(treatment_states), replace=True)
state_to_treatment_year = dict(zip(treatment_states, treatment_years))

    # Add a treatment column to the DataFrame
data['TREATMENT'] = data.apply(lambda x: 1 if x['STATEFIP'] in treatment_states and x['YEAR'] >= state_to_treatment_year[x['STATEFIP']] else 0, axis=1)


    

# Define a function to perform FGLS for each state
def perform_fgls(state_data):
    # Fit an AR(1) model for autocorrelation within the state
    autocorrelation_model = sm.tsa.AR(state_data['residuals']).fit()
    autocorrelation_parameters = autocorrelation_model.params

    # Create a weight matrix for the state based on autocorrelation parameters
    weight_matrix = sm.cov_struct.Autoregressive(lags=1)
    weight_matrix.set_params(params=autocorrelation_parameters)

    # Perform GLS for the state

    X = data[[ 'TREATMENT', 'STATEFIP_2',
       'STATEFIP_4', 'STATEFIP_5', 'STATEFIP_6', 'STATEFIP_8', 'STATEFIP_9',
       'STATEFIP_10', 'STATEFIP_12', 'STATEFIP_13', 'STATEFIP_15',
       'STATEFIP_16', 'STATEFIP_17', 'STATEFIP_18', 'STATEFIP_19',
       'STATEFIP_20', 'STATEFIP_21', 'STATEFIP_22', 'STATEFIP_23',
       'STATEFIP_24', 'STATEFIP_25', 'STATEFIP_26', 'STATEFIP_27',
       'STATEFIP_28', 'STATEFIP_29', 'STATEFIP_30', 'STATEFIP_31',
       'STATEFIP_32', 'STATEFIP_33', 'STATEFIP_34', 'STATEFIP_35',
       'STATEFIP_36', 'STATEFIP_37', 'STATEFIP_38', 'STATEFIP_39',
       'STATEFIP_40', 'STATEFIP_41', 'STATEFIP_42', 'STATEFIP_44',
       'STATEFIP_45', 'STATEFIP_46', 'STATEFIP_47', 'STATEFIP_48',
       'STATEFIP_49', 'STATEFIP_50', 'STATEFIP_51', 'STATEFIP_53',
       'STATEFIP_54', 'STATEFIP_55', 'STATEFIP_56',  'YEAR_1981', 'YEAR_1982',
       'YEAR_1983', 'YEAR_1984', 'YEAR_1985', 'YEAR_1986', 'YEAR_1987',
       'YEAR_1988', 'YEAR_1989', 'YEAR_1990', 'YEAR_1991', 'YEAR_1992',
       'YEAR_1993', 'YEAR_1994', 'YEAR_1995', 'YEAR_1996', 'YEAR_1997',
       'YEAR_1998', 'YEAR_1999', 'YEAR_2000']] # plus any other control variables
    
    y = state_data['TREATMENT']

    model = sm.GLS(y, X, weight=weight_matrix)
    results = model.fit()

    return results

# Group the data by state
state_groups = data.groupby('state')

# Perform FGLS for each state and store the results
state_results = {}

for state, state_data in state_groups:
    state_results[state] = perform_fgls(state_data)

# Combine autocorrelation structures from each state to create a weight matrix for the entire dataset
combined_weight_matrix = sum(state_results[state].cov_params() for state in state_results.values()) / len(state_results)

# Perform GLS for the entire dataset using the combined weight matrix
X = data[[ 'TREATMENT', 'STATEFIP_2',
       'STATEFIP_4', 'STATEFIP_5', 'STATEFIP_6', 'STATEFIP_8', 'STATEFIP_9',
       'STATEFIP_10', 'STATEFIP_12', 'STATEFIP_13', 'STATEFIP_15',
       'STATEFIP_16', 'STATEFIP_17', 'STATEFIP_18', 'STATEFIP_19',
       'STATEFIP_20', 'STATEFIP_21', 'STATEFIP_22', 'STATEFIP_23',
       'STATEFIP_24', 'STATEFIP_25', 'STATEFIP_26', 'STATEFIP_27',
       'STATEFIP_28', 'STATEFIP_29', 'STATEFIP_30', 'STATEFIP_31',
       'STATEFIP_32', 'STATEFIP_33', 'STATEFIP_34', 'STATEFIP_35',
       'STATEFIP_36', 'STATEFIP_37', 'STATEFIP_38', 'STATEFIP_39',
       'STATEFIP_40', 'STATEFIP_41', 'STATEFIP_42', 'STATEFIP_44',
       'STATEFIP_45', 'STATEFIP_46', 'STATEFIP_47', 'STATEFIP_48',
       'STATEFIP_49', 'STATEFIP_50', 'STATEFIP_51', 'STATEFIP_53',
       'STATEFIP_54', 'STATEFIP_55', 'STATEFIP_56',  'YEAR_1981', 'YEAR_1982',
       'YEAR_1983', 'YEAR_1984', 'YEAR_1985', 'YEAR_1986', 'YEAR_1987',
       'YEAR_1988', 'YEAR_1989', 'YEAR_1990', 'YEAR_1991', 'YEAR_1992',
       'YEAR_1993', 'YEAR_1994', 'YEAR_1995', 'YEAR_1996', 'YEAR_1997',
       'YEAR_1998', 'YEAR_1999', 'YEAR_2000']]  # Add your independent variables for the entire dataset
y = data['TREATMENT']

model = sm.GLS(y, X, weight=combined_weight_matrix)
results = model.fit()

# Print results for the entire dataset
print(results.summary())
